In [1]:
import itertools
import numpy as np
import random
from copy import deepcopy

import time

In [2]:
def generate_subsets(edges):
    for r in range(len(edges)+1):
        for subset in itertools.combinations(edges, r):
            yield subset

In [3]:
def get_reach_matrix(graph, n):
    reach = np.zeros((n, n))
    np.copyto(reach, graph)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                reach[i][j] = reach[i][j] or (reach[i][k] and reach[k][j])
    return reach

In [4]:
def is_equivalent(graph1, graph2):
    reach1 = get_reach_matrix(graph1, graph1.shape[0])
    reach2 = get_reach_matrix(graph2, graph2.shape[0])
    return np.array_equal(reach1, reach2)

In [5]:
def med_brute_force(graph):
    n = graph.shape[0]
    edges = set(zip(*np.where(graph)))
    subsets = generate_subsets(edges)
    min_subset_size = np.inf
    min_graph = graph
    for subset in subsets:
        new_edges = edges - set(subset)
        new_graph = np.zeros((n,n))
        for (i,j) in new_edges:
            new_graph[i][j] = 1
        if is_equivalent(graph, new_graph):
            if len(new_edges) < min_subset_size:
                min_subset_size = len(new_edges)
                min_graph = new_graph
    return min_subset_size, min_graph

In [6]:
def med_greedy(graph):
    current_graph = deepcopy(graph)
    n = graph.shape[0]
    edges = set(zip(*np.where(graph)))
    for (i, j) in edges:
        modified_graph = deepcopy(current_graph)
        modified_graph[i][j] = 1 - modified_graph[i][j]
        if is_equivalent(current_graph, modified_graph):
            current_graph = modified_graph
    edges = set(zip(*np.where(current_graph)))
    return len(edges), current_graph


In [7]:
def initialize(graph):
    solution = deepcopy(graph)
    n = graph.shape[0]
    for i in range(n):
        for j in range(n):
            if solution[i][j] == 1:
                if random.random() < 0.2:
                    solution[i][j] = 0
    return np.array(solution)

In [8]:
def calc_solution_value(solution, graph, graph_reach, alpha):
    n = graph.shape[0]
    solution_reach = get_reach_matrix(solution, n)
    
    needed_reachs = 0
    
    for i in range(n):
        for j in range(n):
            if graph_reach[i][j] == 1 and solution_reach[i][j] == 0:
                needed_reachs += 1
            
                
    edges = set(zip(*np.where(solution)))
  
    return len(edges) / n**2 + alpha * needed_reachs     

In [9]:
def shaking(solution, k, edges):
    n = len(edges)
    new_solution = deepcopy(solution)
    
    ns_edges = set(zip(*np.where(new_solution)))
    to_pick = edges - ns_edges
    
    for _ in range(k):
        if len(to_pick) == 0:
            break
        (i, j) = random.choice(tuple(to_pick))
        new_solution[i][j] = 1
    
    return new_solution

In [10]:
def make_small_change(graph, edges):
    new_graph = deepcopy(graph)
    (i, j) = random.choice(tuple(edges))
    new_graph[i][j] = 1 - new_graph[i][j]
    
    return new_graph

In [11]:
def local_search1(solution, graph, n, max_iters, graph_reach, edges, alpha):
    best_solution = deepcopy(solution)
    best_value = calc_solution_value(solution, graph, graph_reach, alpha)
    
    for i in range(max_iters):
        new_solution = make_small_change(best_solution, edges)
        new_value = calc_solution_value(new_solution, graph, graph_reach, alpha)
        
        if new_value < best_value:
            best_value = new_value
            best_solution = deepcopy(new_solution) 
            
    return best_solution, best_value   

In [12]:
def local_search2(solution, graph, n, graph_reach, edges, alpha):
    best_solution = deepcopy(solution)
    best_value = calc_solution_value(solution, graph, graph_reach, alpha)
    
    while True:
        solution_edges = set(zip(*np.where(best_solution)))

        improved = False
        for (i, j) in solution_edges:
            new_solution = deepcopy(best_solution)
            new_solution[i][j] = 0
            new_value = calc_solution_value(new_solution, graph, graph_reach, alpha)
            
            if new_value < best_value:
                best_value = new_value
                best_solution = deepcopy(new_solution)
                improved = True
        
        if not improved:
            break
            
    return best_solution, best_value

In [13]:
def local_search3(solution, graph, n, graph_reach, edges, alpha):
    best_solution = deepcopy(solution)
    best_value = calc_solution_value(solution, graph, graph_reach, alpha)
    
    while True:
        solution_edges = set(zip(*np.where(best_solution)))

        improved = False
        for (i, j) in solution_edges:
            new_solution = deepcopy(best_solution)
            new_solution[i][j] = 0
            
            missing_edges = edges - solution_edges
            me_size = len(missing_edges)
            
            if me_size > 0:
                pos = random.randint(0, me_size-1)
                (row, clm) = list(missing_edges)[pos]

                if random.random() > 0.5:
                    new_solution[row][clm] = 1
            
            new_value = calc_solution_value(new_solution, graph, graph_reach, alpha)
            
            if new_value < best_value:
                best_value = new_value
                best_solution = deepcopy(new_solution)
                improved = True
        
        if not improved:
            break
            
    return best_solution, best_value

In [14]:
def vns(graph, minutes, k_max, move_prob, ls_max_iters, alpha, ls_type):
    solution = initialize(graph)
    n = solution.shape[0]
    graph_reach = get_reach_matrix(graph, n)
    value = calc_solution_value(solution, graph, graph_reach, alpha)
    
    start_time = time.time()
    elapsed_time = 0
    duration_limit = minutes * 60
    
    edges = set(zip(*np.where(graph)))
    
    while elapsed_time < duration_limit:
        for k in range(1, k_max):
            new_solution = shaking(solution, k, edges)
            if ls_type == 1:
                new_solution, new_value = local_search1(new_solution, graph, n, ls_max_iters, graph_reach, edges, alpha)
            elif ls_type == 2:
                new_solution, new_value = local_search2(new_solution, graph, n, graph_reach, edges, alpha)
            elif ls_type == 3:
                new_solution, new_value = local_search3(new_solution, graph, n, graph_reach, edges, alpha)
            
            if new_value < value or (new_value == value and random.random() < move_prob):
                value = new_value
                solution = deepcopy(new_solution)
                break
            elapsed_time = time.time() - start_time
    if value < 1.0:
        value = value * n**2
    else:
        value = -1
    
    return solution, value